# Fire response traits in plants from field samples

This script contains examples of R code to query tables in the database

## Load libraries

In [30]:
library(RPostgreSQL)
library(ggplot2)
library(dplyr)
require(tidyr)
library(galah)
library(ozmaps)
library(sf)
library(data.tree)

In [31]:
here::i_am("read-from-database/Read-table-traits-per-species.ipynb")

here() starts at /Users/z3529065/proyectos/fireveg/fireveg-analysis



In [32]:
galah_config(email = readLines(here::here("secrets","galah-email.txt")))

## Query taxonomy from Atlas of Living Australia

In [10]:
nsw <- st_transform(ozmaps::ozmap_states, 4326) %>% filter(NAME == "New South Wales")

In [11]:
ALA_taxonomy_file <- here::here('data','NSW-plants-according-to-ALA.rds')
if (!file.exists(ALA_taxonomy_file)) {
    plant_families <- galah_call() |>
        galah_identify("plantae") |>
        galah_geolocate(nsw, type = "bbox")|>
        galah_down_to(family) |>
        atlas_taxonomy()
    saveRDS(file=here::here('data','NSW-plants-according-to-ALA.rds'),plant_families)
} else {
    plant_families <- readRDS(ALA_taxonomy_file)
}

In [12]:
print(plant_families, pruneMethod = "dist", limit = 10)

                           levelName
1  Plantae                          
2   ¦--Rhodoplantae                 
3   ¦   °--Glaucophyta              
4   ¦       °--... 1 nodes w/ 2 sub 
5   ¦--Anthocerotophyta             
6   ¦   °--Anthocerotopsida         
7   ¦       °--... 3 nodes w/ 6 sub 
8   ¦--Bryophyta                    
9   ¦   °--... 4 nodes w/ 128 sub   
10  ¦--Charophyta                   
11  ¦   °--... 2 nodes w/ 459 sub   
12  ¦--Marchantiophyta              
13  ¦   ¦--Jungermanniopsida        
14  ¦   ¦   °--... 3 nodes w/ 76 sub
15  ¦   °--... 2 nodes w/ 104 sub   
16  °--Tracheophyta                 
17      °--... 1 nodes w/ 26 sub    


In [13]:
df_rank <- ToDataFrameTypeCol(plant_families, type = "rank")
df_rank[10:20,] |> tibble::as_tibble() |> print(max_footer_lines = 2)

# A tibble: 11 × 10
   rank_kingdom rank_subkingdom rank_phylum rank_class rank_order  rank_subclass
   <chr>        <chr>           <chr>       <chr>      <chr>       <chr>        
 1 Plantae      NA              Bryophyta   Bryopsida  Bryales     Bryidae      
 2 Plantae      NA              Bryophyta   Bryopsida  Hedwigiales Bryidae      
 3 Plantae      NA              Bryophyta   Bryopsida  Hedwigiales Bryidae      
 4 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 5 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 6 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 7 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 8 Plantae      NA              Bryophyta   Bryopsida  Hookeriales Bryidae      
 9 Plantae      NA              Bryophyta   Bryopsida  Hypnales    Bryidae      
10 Plantae      NA              Bryophyta   Bryopsida  Hypnales    Bryidae      
11 Plant

### update table with taxonomy
Add column with the plant order from the taxonomy:

In [14]:
spp.table$rank_order <- df_rank$rank_order[match(spp.table$family,df_rank$rank_family)]

In [15]:
fabales <- c("Fabaceae (Faboideae)", "Fabaceae (Mimosoideae)", "Fabaceae (Caesalpinioideae)" )
unplaced <- c("Dilleniaceae", "Flacourtiaceae")
single_family <- c("Boraginaceae")
rosales <- c("Malaceae")
lamiales <- c("Myoporaceae", "Buddlejaceae")
polypodiales <- c("Pteridaceae")
asparagales <- c("Anthericaceae","Phormiaceae","Lomandraceae", "Agavaceae", "Hyacinthaceae")
ranunculales <- c("Fumariaceae")
malvales <- c("Tiliaceae")


In [16]:
spp.table <- spp.table %>% mutate(
    rank_order = case_when(
            family %in% fabales ~ "Fabales",
            family %in% unplaced ~ "unplaced",
            family %in% rosales ~ "Rosales",
            family %in% lamiales ~ "Lamiales",
            family %in% polypodiales ~ "Polypodiales",
            family %in% asparagales ~ "Asparagales",
            family %in% ranunculales ~ "Ranunculales",
            family %in% malvales ~ "Malvales",
            family %in% single_family ~ sprintf("fam. %s",family),
            is.na(rank_order) ~ "unknown",
            TRUE ~ rank_order
        ))

In [17]:
spp.table %>% 
    filter(rank_order %in% c(NA,"unknown")) %>% 
    group_by(family) %>% 
    summarise(nspp=n()) %>% 
    arrange(desc(nspp))

family,nspp
<chr>,<int>
Unknown Flora,32
Athyriaceae,15
Lemnaceae,13
Viscaceae,13
Stackhousiaceae,12
Characeae,9
Callitrichaceae,8
Dipsacaceae,6
Najadaceae,6


Need to check this again, according to David, there should be around 6000 plant species in NSW, is this list including that many synonyms?

In [39]:
table(spp.table$current)


false  true 
 3511 12221 

In [43]:
spp.table %>%
    #filter(current %in% "true") %>%
    group_by(taxonrank,establishment) %>%
    summarise(total=n(), nspp = n_distinct(spp), ncurrent=n_distinct(current_spp), .groups = "drop") 

taxonrank,establishment,total,nspp,ncurrent
<chr>,<chr>,<int>,<int>,<int>
Cultivar,"Alive in NSW, Native",3,3,3
Cultivar,Introduced,12,12,8
Forma,"Alive in NSW, Native",60,60,51
Forma,Introduced,11,11,6
Hybrid,"Alive in NSW, Native",1,1,1
Nothomorph,Introduced,2,2,2
Species,"Alive in NSW, Native",10231,10231,7943
Species,"Extinct in NSW, Native",38,38,28
Species,Introduced,3255,3255,2763


Examine records for species that are not current:

In [44]:
head(subset(spp.table, current %in% "false"))

,family,genus,spp,species,current_spp,current_species,taxonrank,establishment,current,nquadrat,⋯,repr4,repr2,surv5,surv6,surv7,disp1,repr3a,repr3,surv4,surv1
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7,Poaceae,Anthosachne,2364,Elymus scaber,22952,Anthosachne scabra,Species,"Alive in NSW, Native",false,1,⋯,0,0,0,0,0,0,1,0,1,3
12,Ericaceae,Agiortia,2369,Leucopogon pedicillatus,17241,Agiortia pedicellata,Species,"Alive in NSW, Native",false,0,⋯,0,0,0,0,0,0,0,0,0,0
14,Fabaceae (Faboideae),Abrus,2371,Abrus precatorius,13410,Abrus precatorius subsp. precatorius,Subspecies,Introduced,false,0,⋯,0,0,0,0,0,2,0,0,0,0
15,Poaceae,Paractaenum,2372,Paractaenum novae-hollandiae subsp. reversum,3507,Paractaenum novae-hollandiae,Species,"Alive in NSW, Native",false,0,⋯,0,0,0,0,0,0,0,0,0,0
17,Pteridaceae,Adiantum,2374,Adiantum silvaticum var. silvaticum,7567,Adiantum silvaticum,Species,"Alive in NSW, Native",false,0,⋯,0,0,0,0,0,0,0,0,0,0
28,Plantaginaceae,Linaria,2385,Linaria genistifolia,12682,Linaria dalmatica,Species,Introduced,false,0,⋯,0,0,0,0,0,0,0,0,0,0


In [45]:
spp.table %>% slice(1:5)

family,genus,spp,species,current_spp,current_species,taxonrank,establishment,current,nquadrat,⋯,repr4,repr2,surv5,surv6,surv7,disp1,repr3a,repr3,surv4,surv1
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Brassicaceae,Lepidium,2358,Lepidium oxytrichum,2358,Lepidium oxytrichum,Species,"Alive in NSW, Native",true,0,⋯,0,0,0,0,0,3,0,0,0,1
Myrtaceae,Eucalyptus,2359,Eucalyptus williamsiana,2359,Eucalyptus williamsiana,Species,"Alive in NSW, Native",true,0,⋯,0,0,0,0,0,0,0,0,0,6
Myrtaceae,Melaleuca,2360,Melaleuca glomerata,2360,Melaleuca glomerata,Species,"Alive in NSW, Native",true,0,⋯,0,0,0,0,0,0,0,0,0,3
Apiaceae,Actinotus,2361,Actinotus helianthi,2361,Actinotus helianthi,Species,"Alive in NSW, Native",true,0,⋯,1,0,2,0,1,3,0,3,0,14
Apiaceae,Apium,2362,Apium prostratum,2362,Apium prostratum,Species,"Alive in NSW, Native",true,0,⋯,0,0,0,0,0,1,0,0,0,1


## Literature vs. field work data
Let's check how many valid species have information from both sources


In [47]:
 spp.table %>% 
    filter(
        #current %in% "true", 
           taxonrank %in% "Species",
           establishment %in% "Alive in NSW, Native") %>%
    mutate(
     fieldwork_sources =  nquadrat>0,
     literature_sources = (disp1 +
                           germ1 + germ8 + 
                           rect2 + 
                           grow1 + 
                           repr2 + repr3a + repr3 + repr4 +  
                           surv1 + surv4 + surv5 + surv6 + surv7) > 0
    ) %>% 
    group_by(fieldwork_sources,literature_sources) %>% 
    summarise(total = n_distinct(species), total_current = n_distinct(current_species), .groups = "drop")

fieldwork_sources,literature_sources,total,total_current
<lgl>,<lgl>,<int>,<int>
FALSE,FALSE,4504,3767
FALSE,TRUE,4812,4509
TRUE,FALSE,97,97
TRUE,TRUE,776,774


In [48]:
4509+97+774

[1] 5380

In [24]:
sum(spp.table$repr2>0)

[1] 138

In [25]:
saveRDS(file=here::here('data','Summary-traits-species.rds'),spp.table)